# JIT Engine: Tensor Summation

This example will go over how to compile MLIR code to a function callable from Python.

The example MLIR code we’ll use here performs tensor summation.

Let’s first import some necessary modules and generate an instance of our JIT engine.

In [8]:
import mlir_graphblas
import numpy as np

engine = mlir_graphblas.MlirJitEngine()

We'll use the same set of passes to optimize and compile all of our examples below.

In [2]:
passes = [
    "--graphblas-structuralize",
    "--graphblas-optimize",
    "--graphblas-lower",
    "--sparsification",
    "--sparse-tensor-conversion",
    "--linalg-bufferize",
    "--func-bufferize",
    "--tensor-constant-bufferize",
    "--tensor-bufferize",
    "--finalizing-bufferize",
    "--convert-linalg-to-loops",
    "--convert-scf-to-std",
    "--convert-memref-to-llvm",
    "--convert-math-to-llvm",
    "--convert-openmp-to-llvm",
    "--convert-arith-to-llvm",
    "--convert-math-to-llvm",
    "--convert-std-to-llvm",
    "--reconcile-unrealized-casts"
]

We'll use this MLIR code. 

In [3]:
mlir_text = """
#trait_sum_reduction = {
  indexing_maps = [
    affine_map<(i,j,k) -> (i,j,k)>,
    affine_map<(i,j,k) -> ()>
  ],
  iterator_types = ["reduction", "reduction", "reduction"]
}

func @tensor_sum(%argA: tensor<2x3x5xf32>) -> f32 {
  %output_storage = arith.constant dense<0.0> : tensor<f32>
  %reduction = linalg.generic #trait_sum_reduction
    ins(%argA: tensor<2x3x5xf32>)
    outs(%output_storage: tensor<f32>) {
      ^bb(%a: f32, %x: f32):
        %0 = arith.addf %x, %a : f32
        linalg.yield %0 : f32
  } -> tensor<f32>
  %answer = tensor.extract %reduction[] : tensor<f32>
  return %answer : f32
}"""

Let's compile our MLIR code. 

In [4]:
engine.add(mlir_text, passes)

['tensor_sum']

Let's try out our compiled function. 

In [5]:
# grab our callable
tensor_sum = engine.tensor_sum

# generate inputs
a = np.arange(30, dtype=np.float32).reshape([2,3,5])

# generate output
result = tensor_sum(a)

In [6]:
result

435.0

Let's verify that our function works as expected.

In [7]:
result == a.sum()

True

The examples up to this point make it so that we now know how to perform element-wise operations and reduction operations (e.g. summation). 

With this knowledge, it's fairly straightforward to implement matrix multiplication and dot product calculation. We'll leave this as an exercise for the reader. If help is needed, it's useful to know that matrix multiplication and dot product calculation have already been directly implemented in the `linalg` dialect:

- [linalg.matmul](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgmatmul-mlirlinalgmatmulop)
- [linalg.matmul_i8_i8_i32](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgmatmul_i8_i8_i32-mlirlinalgmatmuli8i8i32op)
- [linalg.matmul_i32_i32_i32](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgmatmul_i32_i32_i32-mlirlinalgmatmuli32i32i32op)
- [linalg.matmul_i16_i16_i32](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgmatmul_i16_i16_i32-mlirlinalgmatmuli16i16i32op)
- [linalg.matmul_column_major](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgmatmul_column_major-mlirlinalgmatmulcolumnmajoropx)
- [linalg.batch_matmul](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgbatch_matmul-mlirlinalgbatchmatmulop)
- [linalg.batch_matmul_i8_i8_i32](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgbatch_matmul_i8_i8_i32-mlirlinalgbatchmatmuli8i8i32op)
- [linalg.batch_matmul_i32_i32_i32](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgbatch_matmul_i32_i32_i32-mlirlinalgbatchmatmuli32i32i32op)
- [linalg.batch_matmul_i16_i16_i32](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgbatch_matmul_i16_i16_i32-mlirlinalgbatchmatmuli16i16i32op)
- [linalg.dot](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgdot-mlirlinalgdotop)
- [linalg.dot_i8_i8_i32](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgdot_i8_i8_i32-mlirlinalgdoti8i8i32op)
- [linalg.dot_i32_i32_i32](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgdot_i32_i32_i32-mlirlinalgdoti32i32i32op)
- [linalg.dot_i16_i16_i32](https://mlir.llvm.org/docs/Dialects/Linalg/#linalgdot_i16_i16_i32-mlirlinalgdoti16i16i32op)

One useful skill worth practicing is seeing how these `linalg` operations lower to see what's going on and comparing those to how we might've implemented things. The MLIR explorer can come in handy for this purpose. 